In [2]:
import cv2, typing, math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
FACE_CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

EYE_CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [4]:
def __detect(classifier: cv2.CascadeClassifier, grayscale: np.ndarray, scale_factor: float, min_neighbors: int, min_size: typing.Tuple[int, int]) -> np.ndarray:
    detections = classifier.detectMultiScale(
        grayscale,
        scaleFactor = scale_factor,
        minNeighbors = min_neighbors,
        minSize = min_size
    )

    return detections.tolist()

In [5]:
def __detect_face(classifier: cv2.CascadeClassifier, grayscale: np.ndarray) -> typing.Tuple[np.ndarray,np.ndarray]:
    SCALE_FACTOR = 1.1
    MIN_NEIGHBORS = 5
    MIN_SIZE = (120, 120)
    return __detect(classifier, grayscale, scale_factor = SCALE_FACTOR, min_neighbors = MIN_NEIGHBORS, min_size = MIN_SIZE)

In [6]:
def __detect_eyes(classifier: cv2.CascadeClassifier, grayscale: np.ndarray) -> np.ndarray:
    SCALE_FACTOR = 1.1
    MIN_NEIGHBORS = 10
    MIN_SIZE = (20, 20)

    return __detect(classifier, grayscale, scale_factor = SCALE_FACTOR, min_neighbors = MIN_NEIGHBORS, min_size = MIN_SIZE)

In [7]:
def __crop_face(image: np.ndarray, face: np.ndarray) -> np.ndarray:
    crops = []
    
    for (x, y, w, h) in face:
    
        crop = image[y:y+h, x:x+w]
    
        crops.append(crop)

    return crops[0]

In [8]:
def __crop_eyes(image: np.ndarray, eyes:np.ndarray) -> np.ndarray:
    crops = []

    for (ex, ey, ew, eh) in eyes:
        crop = image[ey:ey+eh, ex:ex+ew]
        crops.append(crop)

    return crops

In [9]:
def __extract_pupil(eye_crop: np.ndarray) -> np.ndarray:
    
    gray = cv2.cvtColor(eye_crop, cv2.COLOR_RGBA2GRAY if eye_crop.shape[2] == 4 else cv2.COLOR_RGB2GRAY)

    blur = cv2.GaussianBlur(gray, (7, 7), 0)

    # Fix ???
    # 50
    _, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return np.zeros_like(eye_crop) 

    pupil_contour = max(contours, key=cv2.contourArea)

    
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [pupil_contour], -1, 255, -1)

    
    pupil_masked = cv2.bitwise_and(eye_crop, eye_crop, mask=mask)

    return pupil_masked

In [ ]:
image = cv2.imread('./dataset/face1.png')
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGBA)
face = __detect_face(FACE_CASCADE, image)
(x,y,h,w) = face[0]
face = __crop_face(image,face)
eyes = __detect_eyes(EYE_CASCADE,face[0:int(h * 0.6), :])

# crop
eye_crops = __crop_eyes(face,eyes)

# extract pupil
eye_l = __extract_pupil(eye_crops[0])
eye_r = __extract_pupil(eye_crops[1])

#
# show
#
fig, axes = plt.subplots(1, 2, figsize=(6, 3))

axes[0].imshow(eye_r)
axes[1].imshow(eye_l)
plt.show()